In [2]:
import torch
import torch.nn as nn
import torch.optim as opt
torch.set_printoptions(linewidth=120)
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from itertools import product

def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size = 2, stride = 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size = 2, stride = 2)
        x = torch.flatten(x,start_dim = 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)

        return x

In [4]:
train_set = torchvision.datasets.FashionMNIST(root="./data",
train = True,
 download=True,
transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(train_set,batch_size = 100, shuffle = True)

In [5]:
tb = SummaryWriter()
model = CNN()
images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)
tb.add_image("images", grid)
tb.add_graph(model, images)
tb.close()

In [6]:
#Højreklik på runs og ændre stinavn
%load_ext tensorboard 
%tensorboard --logdir C:/Users/yusuf/Documents/GitHub/DeepLearning/runs --host localhost --port 6009

Reusing TensorBoard on port 6008 (pid 9780), started 1:36:04 ago. (Use '!kill 9780' to kill it.)

In [9]:
device = ("cuda" if torch.cuda.is_available() else cpu)
model = CNN().to(device)
train_loader = torch.utils.data.DataLoader(train_set,batch_size = 100, shuffle = True)
optimizer = opt.Adam(model.parameters(), lr= 0.01)
criterion = torch.nn.CrossEntropyLoss()

tb = SummaryWriter()

for epoch in range(10):

    total_loss = 0
    total_correct = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        preds = model(images)

        loss = criterion(preds, labels)
        total_loss+= loss.item()
        total_correct+= get_num_correct(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    tb.add_scalar("Loss", total_loss, epoch)
    tb.add_scalar("Correct", total_correct, epoch)
    tb.add_scalar("Accuracy", total_correct/ len(train_set), epoch)

    tb.add_histogram("conv1.bias", model.conv1.bias, epoch)
    tb.add_histogram("conv1.weight", model.conv1.weight, epoch)
    tb.add_histogram("conv2.bias", model.conv2.bias, epoch)
    tb.add_histogram("conv2.weight", model.conv2.weight, epoch)

    print("epoch:", epoch, "total_correct:", total_correct, "loss:",total_loss)

tb.close()

epoch: 0 total_correct: 47296 loss: 332.8152264356613
epoch: 1 total_correct: 51455 loss: 232.21351762115955
epoch: 2 total_correct: 52066 loss: 216.0624998435378
epoch: 3 total_correct: 52404 loss: 206.31701576709747
epoch: 4 total_correct: 52733 loss: 197.33242172002792
epoch: 5 total_correct: 52944 loss: 191.50036976486444
epoch: 6 total_correct: 52853 loss: 192.47740879654884
epoch: 7 total_correct: 53136 loss: 186.30957846343517
epoch: 8 total_correct: 53279 loss: 183.6894169896841
epoch: 9 total_correct: 53215 loss: 183.22876899689436


In [13]:
for name, weight in model.named_parameters():
    tb.add_histogram(name,weight, epoch)
    tb.add_histogram(f'{name}.grad',weight.grad, epoch)

In [14]:
from itertools import product
parameters = dict(
    lr = [0.01, 0.001],
    batch_size = [32,64,128],
    shuffle = [True, False]
)

param_values = [v for v in parameters.values()]
print(param_values)

for lr,batch_size, shuffle in product(*param_values):
    print(lr, batch_size, shuffle)

[[0.01, 0.001], [32, 64, 128], [True, False]]
0.01 32 True
0.01 32 False
0.01 64 True
0.01 64 False
0.01 128 True
0.01 128 False
0.001 32 True
0.001 32 False
0.001 64 True
0.001 64 False
0.001 128 True
0.001 128 False


In [12]:
for run_id, (lr,batch_size, shuffle) in enumerate(product(*param_values)):
    print("run id:", run_id + 1)
    model = CNN().to(device)
    train_loader = torch.utils.data.DataLoader(train_set,batch_size = batch_size, shuffle = shuffle)
    optimizer = opt.Adam(model.parameters(), lr= lr)
    criterion = torch.nn.CrossEntropyLoss()
    comment = f' batch_size = {batch_size} lr = {lr} shuffle = {shuffle}'
    tb = SummaryWriter(comment=comment)
    for epoch in range(5):
        total_loss = 0
        total_correct = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            preds = model(images)

            loss = criterion(preds, labels)
            total_loss+= loss.item()
            total_correct+= get_num_correct(preds, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        tb.add_scalar("Loss", total_loss, epoch)
        tb.add_scalar("Correct", total_correct, epoch)
        tb.add_scalar("Accuracy", total_correct/ len(train_set), epoch)

        print("batch_size:",batch_size, "lr:",lr,"shuffle:",shuffle)
        print("epoch:", epoch, "total_correct:", total_correct, "loss:",total_loss)
    print("__________________________________________________________")

    tb.add_hparams(
            {"lr": lr, "bsize": batch_size, "shuffle":shuffle},
            {
                "accuracy": total_correct/ len(train_set),
                "loss": total_loss,
            },
        )

tb.close()

run id: 1
batch_size: 32 lr: 0.01 shuffle: True
epoch: 0 total_correct: 47504 loss: 1039.1458518356085
batch_size: 32 lr: 0.01 shuffle: True
epoch: 1 total_correct: 50467 loss: 813.7473013177514
batch_size: 32 lr: 0.01 shuffle: True
epoch: 2 total_correct: 50855 loss: 778.6177537478507
batch_size: 32 lr: 0.01 shuffle: True
epoch: 3 total_correct: 51106 loss: 761.7775171101093
batch_size: 32 lr: 0.01 shuffle: True
epoch: 4 total_correct: 51285 loss: 741.3965790607035
__________________________________________________________
run id: 2
batch_size: 32 lr: 0.01 shuffle: False
epoch: 0 total_correct: 48395 loss: 977.1445181667805
batch_size: 32 lr: 0.01 shuffle: False
epoch: 1 total_correct: 51193 loss: 760.1825577318668
batch_size: 32 lr: 0.01 shuffle: False
epoch: 2 total_correct: 51542 loss: 731.6286376416683
batch_size: 32 lr: 0.01 shuffle: False
epoch: 3 total_correct: 51724 loss: 718.7634320557117
batch_size: 32 lr: 0.01 shuffle: False
epoch: 4 total_correct: 51875 loss: 706.418440885